- blog 내용 데이터 크롤링

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from pydantic import BaseModel, Field

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)
title_url_list = ['https://blog.naver.com/jeju8253/223364786121']

In [4]:
bs_url = title_url_list [0]
bs_url

'https://blog.naver.com/jeju8253/223364786121'

In [5]:
m_url = "https://m." + bs_url.replace("https://","") # mobile 버전으로 우회
driver.get(m_url)
driver.implicitly_wait(10)

In [6]:
contents_ele = driver.find_elements(By.CLASS_NAME,'se-main-container')

In [7]:
contents_text = contents_ele[0].text

In [8]:
# Example
contents_text

'비가 아주 많이 내리던 어느 날 제주에서 광주를 통해 1박2일 볼일을 보기 위한 일정이 시작되었습니다\n이때 힘들었던 일이 있어서 우울한 마음과 걱정 등 여러 가지 복합적인 감정을 가지고\n일정을 봤던 게 기억나는데 이게 벌써 작년 여름이었으니 시간 참 빠르다는 생각도 듭니다\n결국 힘든 일들도 시간이 약인 건가 싶기도 하네요\n빛고을떡갈비\n주소: 광주광역시 광산구 광산로29번길 14 송정동831-21\n첫 번째 일정이 시작되기 전 공항에서 단체 버스로 이동을 하여 전라도 광주 맛집 한곳을 찾아가게 되었습니다\n하늘에 구멍 난 것처럼 비가 쏟아지다 보니 일단 식당으로 뛰어 들어갔다가 나오면서 식당 외부 사진을 찍었던 기억이 나는데요\n어르신들이 친구들과 함께 식사를 하러 가셨던 건지 함께 식당에서 나오고 있는 모습도 볼 수 있었답니다\n건강하게 나이를 먹고 저렇게 친구들과 맛있는 거 먹고 소소한 행복을 나누는 삶을 살 수 있다는 것도 행운인 것 같습니다\n1. 깔끔해 보였던 식당 내부\n광주 송정 떡갈비 맛집 인지 아닌지는 먹어봐야 알 수 있겠죠\n그래도 일단 기대는 가득 가지고 식당 내부로 들어가게 되었습니다\n오래전 광주 여행을 했을 때도 떡갈비 식당 찾아갔다가 만족하고 왔던 기억이 있어서 더욱 기대를 했던 것 같아요\n개인적으로 떡갈비보다는 고기 그대로 구워주는 식당을 좋아하긴 하지만 광주 떡갈비 맛집들은\n그래도 다른 지역의 식당들 보다 반찬도 잘 나오고 그래서 만족도가 더 높더라고요\n여행을 다니거나 일상생활 속 식당을 찾아다니면서 사진은 잔뜩 찍어 놓긴 하지만\n막상 올리려고 하면 귀찮기도 하고 한 식당에서 마음먹고 포스팅해볼까 하고 촬영하지 않는 경우는\n생각보다 사진 양이 많지 않기도 해서 그냥 폰이나 카메라에 사진만 보관하게 되는 일도 다반사인데\n그래도 그런 사진을 보관해 놓기라도 하니 이렇게 전남 광주 송정동 맛집 처럼 오랜만에 사진 꺼내 보기도 하고\n추억을 기록하는 시간을 갖게 되기도 하네요\n2. 다양한 밑반찬\n전라도 광주 맛집 

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
import time

import os
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [14]:
# naver 검색
searchkey = "전라남도 광주 여행"
url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={searchkey}"

driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)
driver.get(url)
driver.implicitly_wait(10)

# 스크롤 아래로
actions = driver.find_element(By.CSS_SELECTOR, 'body')
actions.send_keys(Keys.END)

time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)

# 블로그 url 리스트화
title_element_list = driver.find_elements(By.CLASS_NAME,'title_link')

title_url_list =[]
for title_url in title_element_list:
    url_append = title_url.get_attribute('href')
    title_url_list.append(url_append)
driver.close()

print("크롤링 사이트 개수 : ", len(title_url_list))

크롤링 사이트 개수 :  110


In [16]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)

contents_text_list=[]
for bs_url in title_url_list:
    m_url = "https://m." + bs_url.replace("https://","") # mobile 버전으로 우회
    driver.get(m_url)
    driver.implicitly_wait(10)

    # 본문 내용
    contents_ele = driver.find_elements(By.CLASS_NAME,'se-main-container')
    try:
        contents_text = contents_ele[0].text
        contents_text_list.append(contents_text)
    except Exception as e:
        print(e)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [18]:
len(contents_text_list)

104

In [20]:
# 내용을 list save
import pickle

with open("./DATA/전라남도_광주_여행_contents_list.pkl","wb") as f:
    pickle.dump(contents_text_list, f)